In [4]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
import torch,cv2
from fastai.conv_learner import *
PATH = "data/market1501/"
os.makedirs(PATH,exist_ok=True)

In [6]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [12]:
ls {PATH}

gt_bbox/  gt_query/  models/  query/  readme.txt  test/  train/


In [13]:
from fastai.plots import *

In [14]:
def get_1st(path): return glob(f'{path}/*.*')[0]

In [ ]:
dc_path = "data/market1501/train/"
list_paths = [get_1st(f"{dc_path}cats"), get_1st(f"{dc_path}dogs")]
plots_from_files(list_paths, titles=["cat", "dog"], maintitle="Single-label classification")

In [9]:
bs = 16
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [10]:
data = get_data(32,4)

NotADirectoryError: [Errno 20] Not a directory: 'data/market1501/train/0002_c1s1_000451_03.jpg'

In [ ]:
x,y = next(iter(data.trn_dl))

In [ ]:
plt.imshow(data.trn_ds.denorm(x)[0])

In [ ]:
data = get_data(32,bs)

In [ ]:
lr = 1e-2

In [15]:
class BnLayer(nn.Module):
    def __init__(self,ni,nf,stride=2,kernel_size=3):
        super().__init__()
        self.conv = nn.Conv2d(ni,nf,stride=stride,kernel_size=kernel_size,padding=1,bias=False)
        self.a = nn.Parameter(torch.zeros(nf,1,1))
        self.m = nn.Parameter(torch.ones(nf,1,1))
        
    def forward(self,x):
        x = F.relu(self.conv(x))
        x_chan = x.transpose(0,1).contiguous().view(x.size(1), -1)
        if self.training:
            self.means = x_chan.mean(1)[:,None,None]
            self.stds  = x_chan.std (1)[:,None,None]
        return (x-self.means)/self.stds*self.a*self.m

In [16]:
class ResNetLayer(BnLayer):
    def forward(self,x):
        return x + super().forward(x)

In [17]:
class lwfNet(nn.Module):
    def __init__(self,layers,c,p=0.5):
        super().__init__()
        self.conv1 = BnLayer(3,16,stride=1,kernel_size=3)
        self.layers = nn.ModuleList([BnLayer(layers[i],layers[i+1]) 
                                     for i in range(len(layers)-1)])
        self.layer2 = nn.ModuleList([ResNetLayer(layers[i],layers[i+1],1)
                                    for i in range(len(layers)-1)])
        self.layer3 = nn.ModuleList([ResNetLayer(layers[i],layers[i+1],1)
                                    for i in range(len(layers)-1)])
        self.out = nn.Linear(layers[-1],c)
        self.drop = nn.Dropout(p)
        
    def forward(self,x):
        x = self.conv(x)
        for l,l2,l3 in zip(self.layers,self.layers2,self.layers3):
            x = l3(l2(l(x)))
        x = F.adaptive_max_pool2d(x,1)
        x = x.view(x.size(0),-1)
        x = self.drop(x)
        return F.log_softmax(self.out(x),dim=-1)